In [1]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import math
import random

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images / 255
test_images = test_images / 255



new_train_images = np.zeros((10000,280,280))
new_train_labels = np.zeros((10000,14))

for i in range(len(new_train_images)):

    randx = random.randrange(0,280-28,1)
    randy = random.randrange(0,280-28,1)

    new_train_images[i,randx:randx+28,randy:randy+28] = train_images[i]
    new_train_labels[i,0] = randx
    new_train_labels[i,1] = randy 
    new_train_labels[i,2] = randx + 28
    new_train_labels[i,3] = randy + 28
    new_train_labels[i,4+train_labels[i]] = 1 



trainimagestensor = tf.convert_to_tensor(new_train_images, dtype=tf.float16)
trainlabelstensor = tf.convert_to_tensor(new_train_labels, dtype=tf.float16)




2023-11-06 17:52:47.213325: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-06 17:52:47.319521: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F AVX512_VNNI AVX512_BF16, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-06 17:52:53.070511: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-06 17:52:53.153425: I t

In [33]:
class CustomLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()

    def call(self, y_true, y_pred):        
        
        truebox = y_true[0:4]
        truelabel = y_true[4:]

        predbox = y_pred[0:4]
        predlabel = y_pred[4:]
        
        log_y_pred = tf.math.log(predlabel)
        elements = -tf.math.multiply_no_nan(x=log_y_pred, y=truelabel)
        ce = tf.reduce_mean(tf.reduce_sum(elements,axis=1))
        bbe = (truebox[0]-predbox[0])**2 + (truebox[1]-predbox[1])**2 + (truebox[2]-predbox[2])**2 + (truebox[3]-predbox[3])**2

        return ce + bbe

def customactivation(inputs):
    
    answers = [0]*len(inputs)
    denom = 0

    for i in range(10):
        denom += math.exp(inputs[i+4])
    for i in range(10):
        answers[i] = math.exp(inputs[i+4])/denom
    for i in range(4):
        answers[i] = inputs[i]
    return answers




model = tf.keras.models.Sequential()


model.add(tf.keras.layers.Conv2D(filters=2,kernel_size=(3,3),strides=1,padding="same",activation="relu", input_shape=(280,280,1)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation = "relu"))
model.add(tf.keras.layers.Dense(14))
model.add(tf.keras.layers.Lambda(customactivation))
model.summary()




model.compile(
  optimizer=tf.keras.optimizers.Adam(0.001),
  loss=CustomLoss(),
  metrics=['accuracy'])


TypeError: Exception encountered when calling layer "lambda_1" (type Lambda).

len is not well defined for a symbolic Tensor (Placeholder:0). Please call `x.shape` rather than `len(x)` for shape information.

Call arguments received by layer "lambda_1" (type Lambda):
  • inputs=tf.Tensor(shape=(None, 14), dtype=float32)
  • mask=None
  • training=None

In [30]:
model.fit(trainimagestensor, trainlabelstensor, epochs=10)

Epoch 1/10
313/313 [==============================] - 4s 11ms/step - loss: 28814.9688 - accuracy: 0.7560
Epoch 2/10
313/313 [==============================] - 3s 10ms/step - loss: 28417.6328 - accuracy: 0.8164
Epoch 3/10
313/313 [==============================] - 3s 10ms/step - loss: 28706.0098 - accuracy: 0.8584
Epoch 4/10
313/313 [==============================] - 3s 10ms/step - loss: 29351.8711 - accuracy: 0.8977
Epoch 5/10
313/313 [==============================] - 3s 10ms/step - loss: 29699.4746 - accuracy: 0.9258
Epoch 6/10
313/313 [==============================] - 3s 10ms/step - loss: 29132.2344 - accuracy: 0.9384
Epoch 7/10
313/313 [==============================] - 3s 10ms/step - loss: 29745.8340 - accuracy: 0.9412
Epoch 8/10
313/313 [==============================] - 3s 10ms/step - loss: 28981.0566 - accuracy: 0.9550
Epoch 9/10
313/313 [==============================] - 3s 10ms/step - loss: 29344.0664 - accuracy: 0.9537
Epoch 10/10
313/313 [==============================] - 

In [31]:
model.predict(trainimagestensor)[0]


313/313 [==============================] - 1s 4ms/step


array([2.1818532e-01, 2.3445489e-01, 2.7069104e-01, 2.7470291e-01,
       1.8443093e-04, 2.1854487e-04, 1.9149730e-04, 1.9640278e-04,
       2.1361448e-04, 1.9754932e-04, 1.8564430e-04, 2.0710978e-04,
       1.9064895e-04, 1.8043096e-04], dtype=float32)

In [7]:
trainlabelstensor[0]

<tf.Tensor: shape=(14,), dtype=float16, numpy=
array([115., 121., 143., 149.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,
         0.,   0.,   0.], dtype=float16)>